In [1]:
from typing import List
import numpy as np
import pandas as pd

In [2]:
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})

In [3]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [4]:
#numpy is the library serving as a bridge between Pandas and the modelling libraries.
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [5]:
data.to_numpy()

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [6]:
#converting back to a DataFrame
pd.DataFrame(data.to_numpy())

,0,1,2
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [7]:
#converting back to a DataFrame: you can pass a numpy array with column names
df2 = pd.DataFrame(data.to_numpy(), columns=['one', 'two', 'three'])

In [8]:
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [9]:
#The to_numpy method works when data is homogenous. If the data is hererogenous,
#then the result is we get an ndarray consisting of Python objects.
df3 = data.copy()

In [10]:
df3['strings'] = ['a', 'b', 'c', 'd', 'e']

In [11]:
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [12]:
df3.to_numpy()

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [13]:
#for some models, we use only a subset of the columns. Recommended: use .iloc indexing with .to_numpy()
model_cols: List[str] = ['x0', 'x1']

In [14]:
data.loc[:, model_cols].to_numpy()

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [15]:
#addressing nonnumeric columns
#remember the pd.Categorical type (pg. 235) and pd.get_dummies
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
                                 categories=['a', 'b'])

In [16]:
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [17]:
#Replacing the category column with dummy variables:
#1) Create dummy variables with pd.get_dummies
#2) Drop the category column
#3) Join the result
dummies = pd.get_dummies(data.category, prefix='category')

In [18]:
dummies

,category_a,category_b
0,1,0
1,0,1
2,1,0
3,1,0
4,0,1


In [19]:
data_with_dummies = data.drop('category', axis=1).join(dummies)

In [20]:
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


In [21]:
#how do we convert the category back into a single column
pd.from_dummies(data_with_dummies[['category_a', 'category_b']])
#we still have the category prefix

,
0,category_a
1,category_b
2,category_a
3,category_a
4,category_b


In [22]:
pd.from_dummies(data_with_dummies[['category_a', 'category_b']], sep='_') #sep removes the prefix

,category
0,a
1,b
2,a
3,a
4,b


In [23]:
#patsy.dmatries function takes a formula string along with a dataset and produces
#design matrices for a particular model
data = pd.DataFrame({'x0': [1, 2, 3, 4, 5],
                    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
                    'y': [-1.5, 0., 3.6, 1.3, -2.]})

In [24]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [25]:
import patsy

In [26]:
y, X = patsy.dmatrices('y ~ x0 + x1', data, )

In [27]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [28]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [29]:
#these patsy DesignMatrix instances are numpy ndarrays with additional metadata
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [30]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [31]:
#the intercept in X came from models like OLS, but we can remove it by adding + 0 to the model
#before removing intercept
patsy.dmatrices('y ~ x0 + x1', data)[1]

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [32]:
#after removing intercept
patsy.dmatrices('y ~ x0 + x1', data)[1]

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [33]:
#We can pass patsy objects directly into numpy algorithms like numpy.linalg.lstsq
#numpy.linalg.lstsq performs an ordinary least squares regression
coef, resid, _, _ = np.linalg.lstsq(X, y, rcond=None)

In [34]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [35]:
#Model metadata retained in the design-info attribute enabling reattachment of the model columns
pd.Series(coef.squeeze())

0    0.312910
1   -0.079106
2   -0.265464
dtype: float64

In [36]:
#Model metadata retained in the design-info attribute enabling reattachment of the model columns
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)

In [37]:
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

In [38]:
#data transformations in patsy formulas
#we can mix python code into patsy formulas
#the library tries to find functions we use in the enclosing scope
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)

In [39]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [40]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [41]:
y, X = patsy.dmatrices('y ~ x0**2 + 0', data)

In [42]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [43]:
X

DesignMatrix with shape (5, 1)
  x0
   1
   2
   3
   4
   5
  Terms:
    'x0' (column 0)

In [44]:
#some commonly used variable transformations:
#1) Standardizing: to mean 0 and variance 1
#2) Centering: subtracting the mean
y, X = patsy.dmatrices('y ~ standardize(x0) * center(x1)', data)

In [45]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [46]:
X

DesignMatrix with shape (5, 4)
  Intercept  standardize(x0)  center(x1)  standardize(x0):center(x1)
          1         -1.41421        0.78                    -1.10309
          1         -0.70711        0.76                    -0.53740
          1          0.00000        1.02                     0.00000
          1          0.70711       -3.33                    -2.35467
          1          1.41421        0.77                     1.08894
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)
    'standardize(x0):center(x1)' (column 3)

In [47]:
#we might fit a motel to one dataset, then evaluate it based on another
#be careful making predictions based on new data
#patsy.build_design_matrices
new_data = pd.DataFrame({'x0': [6, 7, 8, 9],
                        'x1': [3.1, -0.5, 0, 2.3],
                        'y': [1, 2, 3, 4]})

In [48]:
new_X = patsy.build_design_matrices([X.design_info], new_data)

In [49]:
new_X

[DesignMatrix with shape (4, 4)
   Intercept  standardize(x0)  center(x1)  standardize(x0):center(x1)
           1          2.12132        3.87                     8.20951
           1          2.82843        0.27                     0.76368
           1          3.53553        0.77                     2.72236
           1          4.24264        3.07                    13.02491
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)
     'standardize(x0):center(x1)' (column 3)]

In [50]:
np.asarray(new_X)

array([[[ 1.        ,  2.12132034,  3.87      ,  8.20950973],
        [ 1.        ,  2.82842712,  0.27      ,  0.76367532],
        [ 1.        ,  3.53553391,  0.77      ,  2.72236111],
        [ 1.        ,  4.24264069,  3.07      , 13.02490691]]])

In [51]:
np.asarray(new_X[0])

array([[ 1.        ,  2.12132034,  3.87      ,  8.20950973],
       [ 1.        ,  2.82842712,  0.27      ,  0.76367532],
       [ 1.        ,  3.53553391,  0.77      ,  2.72236111],
       [ 1.        ,  4.24264069,  3.07      , 13.02490691]])

In [52]:
a = np.asarray(new_X[0])
a

array([[ 1.        ,  2.12132034,  3.87      ,  8.20950973],
       [ 1.        ,  2.82842712,  0.27      ,  0.76367532],
       [ 1.        ,  3.53553391,  0.77      ,  2.72236111],
       [ 1.        ,  4.24264069,  3.07      , 13.02490691]])

In [53]:
manual_result = (a[:,1] * a[:, 2]).T #second column multiplied by first is the same as the final column

In [54]:
assert all(manual_result == a[:,3]) #the custom result is the same as column 3

In [55]:
#+ sign does not mean addition in patsy, so we have to make sure that we use the correct expression
#which is I(item0 + item1) means item0+item1 
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)

In [56]:
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

In [57]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [58]:
print(data_arr := np.asarray(data))

[[ 1.    0.01 -1.5 ]
 [ 2.   -0.01  0.  ]
 [ 3.    0.25  3.6 ]
 [ 4.   -4.1   1.3 ]
 [ 5.    0.   -2.  ]]


In [59]:
custom_y = data_arr[:,0].reshape(5, 1) + data_arr[:,1].reshape(5, 1)

In [60]:
custom_arr0 = np.concatenate([data_arr, custom_y], axis=1)

In [61]:
#custom_arr0 is my replication of the model's results
custom_arr0

array([[ 1.  ,  0.01, -1.5 ,  1.01],
       [ 2.  , -0.01,  0.  ,  1.99],
       [ 3.  ,  0.25,  3.6 ,  3.25],
       [ 4.  , -4.1 ,  1.3 , -0.1 ],
       [ 5.  ,  0.  , -2.  ,  5.  ]])

In [62]:
#Patsy with Categorical data

#Nonnumeric terms are converted to dummy variables by default
#If there's an intercept then a level will be left out to avoid colinearity
data = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
                    'key2': np.concatenate([np.tile([0, 1], 3), np.array([0, 0])]),
                    'v1': np.arange(1, 9),
                    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]})
#note: we made the Dictionary above using other ways of getting the same values than copying
#from the book

In [63]:
data

,key1,key2,v1,v2
0,a,0,1,-1.0
1,a,1,2,0.0
2,b,0,3,2.5
3,b,1,4,-0.5
4,a,0,5,4.0
5,b,1,6,-1.2
6,a,0,7,0.2
7,b,0,8,-1.7


In [64]:
y, X = patsy.dmatrices('v2 ~ key1', data)

In [65]:
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [66]:
#if we omit the intercept data, then cols for each category value will be included
#in the model design matrix
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)

In [67]:
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [68]:
#numeric columns can be interpreted as categorical with the C function
y, X = patsy.dmatrices('v2 ~ C(key2)', data)

In [69]:
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [70]:
#when we are using multiple categorical terms in a model, it can be more complicated
#we can use interactions like key1:key2 in analysis of variance (ANOVA) models
data['key2']

0    0
1    1
2    0
3    1
4    0
5    1
6    0
7    0
Name: key2, dtype: int64

In [71]:
#when we are using multiple categorical terms in a model, it can be more complicated
#we can use interactions like key1:key2 in analysis of variance (ANOVA) models
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})

In [72]:
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [73]:
y, X = patsy.dmatrices('y ~ key1 + key2', data)

In [74]:
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [75]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)

In [76]:
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

In [77]:
np.asarray(X)

array([[1., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 1., 1., 1.],
       [1., 1., 0., 0.],
       [1., 0., 1., 0.],
       [1., 1., 0., 0.],
       [1., 0., 1., 0.],
       [1., 1., 1., 1.]])

In [78]:
test_arr0 = np.where((X[:, 1] == 1) & (X[:, 2] == 1), np.ones_like(X[:, 1]), np.zeros_like(X[:, 1]))
test_arr0

array([0., 0., 1., 0., 0., 0., 0., 1.])

In [79]:
test_arr0 == X[:, -1]

array([ True,  True,  True,  True,  True,  True,  True,  True])